In [1]:
import os
import pickle5 as pickle
import json
import utils
import numpy as np
import pandas as pd
import pyarrow as pa
import networkx as nx

from tqdm import tqdm

from sherlock import helpers
from sherlock.deploy.model import SherlockModel
from sherlock.functional import extract_features_to_csv
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (
    extract_features,
    convert_string_lists_to_lists,
    prepare_feature_extraction,
    load_parquet_values,
)
from sherlock.features.word_embeddings import initialise_word_embeddings

%load_ext autoreload

# Initialize Sherlock Model

In [2]:
prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 4 files:
        
 ../sherlock/features/glove.6B.50d.txt, 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy,
        
 ../sherlock/features/par_vec_trained_400.pkl.trainables.syn1neg.npy, and 
 ../sherlock/features/par_vec_trained_400.pkl.wv.vectors.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Word Embeddings process took 0:00:12.503732 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:47.488976 seconds. (filename = ../sherlock/features/par_vec_trained_400.pkl)


[nltk_data] Downloading package punkt to /home/aristotle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aristotle/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Initialised NLTK, process took 0:00:04.516164 seconds.


In [4]:
# Initialize the pre-trained sherlock model with its default parameters 
model = SherlockModel();
model.initialize_model_from_json(with_weights=True, model_id="sherlock");

# Run Sherlock on the Synthetic Benchmark (SB)

In [6]:
synthetic_benchmark_data_dir="data/synthetic_benchmark/"
graph = pickle.load(open('graph_representations/synthetic_benchmark/bipartite.graph', 'rb'))

# Groundtruth Homographs
gt = pd.read_pickle('ground_truth/synthetic_example_groundtruth_dict.pickle')
gt_homographs = set([val for val in gt if gt[val]=='homograph'])

# Find the semantic types for each column
column_node_to_semantic_type_dict = utils.sherlock_helpers.get_column_node_to_semantic_type_dict(data_dir=synthetic_benchmark_data_dir, model=model)

# Find the semantic types for each cell node
cell_node_to_semantic_type_dict = utils.sherlock_helpers.get_cell_node_to_semantic_type(graph, column_node_to_semantic_type_dict)
predicted_homographs = utils.sherlock_helpers.get_predicted_homographs(cell_node_to_semantic_type_dict)

Extracting Features: 100%|██████████| 3/3 [00:00<00:00,  8.69it/s]

Exporting 1588 column features



  8%|▊         | 1/13 [00:00<00:04,  2.45it/s]

Index(['animal_name', 'animal_scientific_name', 'country'], dtype='object') ['species' 'species' 'country']
Exporting 1588 column features


 23%|██▎       | 3/13 [00:00<00:02,  3.35it/s]

...

Index(['city', 'country'], dtype='object') ['city' 'country']
Exporting 1588 column features


Extracting Features: 100%|██████████| 2/2 [00:00<00:00, 14.89it/s]

Index(['plant_name', 'plant_scientific_name', 'plant_family', 'country'], dtype='object') ['name' 'name' 'family' 'country']
Exporting 1588 column features



100%|██████████| 13/13 [00:04<00:00,  2.76it/s]

Index(['country', 'country_code'], dtype='object') ['country' 'country']


In [9]:
precision = len(gt_homographs & predicted_homographs) / len(predicted_homographs)
recall = len(gt_homographs & predicted_homographs) / len(gt_homographs)
f1_score = (2* precision * recall) / (precision + recall)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)

Precision: 1.0
Recall: 0.6909090909090909
F1-Score: 0.8172043010752689


In [10]:
print("GT homographs not predicted by Sherlock:", gt_homographs - predicted_homographs)

GT homographs not predicted by Sherlock: {'Christophe', 'Nadine', 'Conroy', 'Elan', 'Heather', 'Reid', 'Garvey', 'Duff', 'Charity', 'Smitty', 'Leandra', 'Vinson', 'Jimmy', 'Crossfire', 'Berkeley', 'Costanza', 'Else'}


# Run Sherlock on TUS Benchmark

In [15]:
data_dir="data/TUS/csvfiles/"
graph = pickle.load(open('graph_representations/TUS/bipartite.graph', 'rb'))

# Groundtruth Homographs
with open("ground_truth/groundtruth_TUS_short_format.pickle", "rb") as fh:
    gt = pickle.load(fh)
gt_homographs = set([val for val in gt if gt[val]=='homograph'])

column_node_to_semantic_type_dict = pd.read_pickle('output/TUS/column_node_to_semantic_type_dict.pickle')
cell_node_to_semantic_type_dict = pd.read_pickle('output/TUS/cell_node_to_semantic_type_dict.pickle')
predicted_homographs = pd.read_pickle('output/TUS/predicted_homographs.pickle')

In [19]:
precision = len(gt_homographs & predicted_homographs) / len(predicted_homographs)
recall = len(gt_homographs & predicted_homographs) / len(gt_homographs)
f1_score = (2* precision * recall) / (precision + recall)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)

Precision: 0.7449533939497913
Recall: 0.5003648934127136
F1-Score: 0.5986397683929966


In [28]:
print(len(gt_homographs))
print(len(predicted_homographs))

26035
17487


In [29]:
cell_node_to_semantic_type_dict['Music Faculty']

{'address', 'description', 'location'}

In [30]:
utils.graph_helpers.get_attribute_of_instance(graph, 'Music Faculty')

['CommonName_t_67c3f7ce5eab8804____c10_0____3.csv',
 'Landmark_t_67c3f7ce5eab8804____c11_0____3.csv',
 'Landmark_t_67c3f7ce5eab8804____c11_1____3.csv',
 'CommonName_t_67c3f7ce5eab8804____c12_0____3.csv',
 'Landmark_t_67c3f7ce5eab8804____c12_0____3.csv',
 'CommonName_t_67c3f7ce5eab8804____c12_1____3.csv',
 'Landmark_t_67c3f7ce5eab8804____c12_1____3.csv',
 'CommonName_t_67c3f7ce5eab8804____c13_0____3.csv',
 'Landmark_t_67c3f7ce5eab8804____c13_0____3.csv',
 'CommonName_t_67c3f7ce5eab8804____c13_1____3.csv',
 'Landmark_t_67c3f7ce5eab8804____c13_1____3.csv',
 'CommonName_t_67c3f7ce5eab8804____c14_0____3.csv',
 'CommonName_t_67c3f7ce5eab8804____c14_1____3.csv',
 'Landmark_t_67c3f7ce5eab8804____c14_1____3.csv',
 'CommonName_t_67c3f7ce5eab8804____c15_0____3.csv',
 'Landmark_t_67c3f7ce5eab8804____c15_0____3.csv',
 'CommonName_t_67c3f7ce5eab8804____c15_1____3.csv',
 'Landmark_t_67c3f7ce5eab8804____c15_1____3.csv',
 'CommonName_t_67c3f7ce5eab8804____c16_0____3.csv',
 'Landmark_t_67c3f7ce5eab8804_

# Run Sherlock on TUS-I Benchmark

In [8]:
base_dir="output/TUS_injected_homographs/"
cardinality_list, precision_list, recall_list, f1_score_list = [], [], [], []

for dir in sorted(os.listdir(base_dir)):
    gt_homographs = set()
    for i in range(1,51):
        gt_homographs.add("InjectedHomograph"+str(i))

    cardinality=int(dir.split('_')[1])
    if os.path.isfile(base_dir+dir+'/predicted_homographs.pickle'): 
        predicted_homographs = pd.read_pickle(base_dir+dir+'/predicted_homographs.pickle')

        precision = len(gt_homographs & predicted_homographs) / len(predicted_homographs)
        recall = len(gt_homographs & predicted_homographs) / len(gt_homographs)
        f1_score = (2* precision * recall) / (precision + recall)

        print(cardinality, precision, recall, f1_score)

        cardinality_list.append(cardinality)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1_score)

0 0.008095440988495952 0.76 0.016020236087689717
100 0.005352146940761465 0.88 0.010639584088985613
500 0.006248047485160888 0.8 0.012399256044637324


In [ ]:
print("Cardinality:", cardinality_list)
print("Precision:", precision_list)
print("Recall:", recall_list)
print("F1-Score:", f1_score_list)

In [38]:
# gt_homographs = set()
# for i in range(1,51):
#     gt_homographs.add("InjectedHomograph"+str(i))


# precision = len(gt_homographs & predicted_homographs) / len(predicted_homographs)
# recall = len(gt_homographs & predicted_homographs) / len(gt_homographs)
# f1_score = (2* precision * recall) / (precision + recall)

# print("Precision:", precision)
# print("Recall:", recall)
# print("F1-Score:", f1_score)

Precision: 0.006248047485160888
Recall: 0.8
F1-Score: 0.012399256044637324
